<h1> Lab 2: Custom Python Package </h1>

<h2> Exercise 2: EDA and Baseline Model </h2> 

In [2]:
import alancarrml 

print(alancarrml.__file__)

/home/bened/DataScience/AMLA/labs/adv_mla_lab_2/alancarrml-project/src/alancarrml/__init__.py


In [3]:
import sys 
print(sys.executable) 
print(sys.prefix)

/home/bened/.cache/pypoetry/virtualenvs/alancarrml-R6IIQ6BQ-py3.12/bin/python
/home/bened/.cache/pypoetry/virtualenvs/alancarrml-R6IIQ6BQ-py3.12


# 2. Load and Explore Dataset

In [4]:
%load_ext autoreload 
%autoreload 2

In [5]:
import alancarrml

In [6]:
import pandas as pd 
import numpy as np

In [7]:
import os

project_dir = '/home/bened/DataScience/AMLA/labs/adv_mla_lab_2' 
data_dir = os.path.join(project_dir, "data") 
raw_data_dir = os.path.join(data_dir, 'raw') 
os.listdir(raw_data_dir)
dataset_path = os.path.join(raw_data_dir, 'insurance.csv') 

df = pd.read_csv(dataset_path) 

In [8]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,18,female,33.820,0,no,southeast,1630.66170
1,19,female,23.480,1,no,southeast,1836.80430
2,46,male,30.570,2,no,southeast,6632.35130
3,54,male,32.050,1,yes,southeast,31922.42950
4,21,male,21.345,4,no,northeast,1638.37255


In [9]:
df.shape

(50000, 7)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       50000 non-null  int64  
 1   sex       50000 non-null  object 
 2   bmi       50000 non-null  float64
 3   children  50000 non-null  int64  
 4   smoker    50000 non-null  object 
 5   region    50000 non-null  object 
 6   charges   50000 non-null  float64
dtypes: float64(2), int64(2), object(3)
memory usage: 2.7+ MB


In [11]:
df.describe()

,age,bmi,children,charges
count,50000.000000,50000.000000,50000.000000,50000.000000
mean,39.463120,30.713734,1.113760,13343.216363
std,14.117142,6.092727,1.212835,12131.222744
min,18.000000,17.291000,0.000000,1137.535900
25%,27.000000,26.600000,0.000000,4694.431800
50%,40.000000,30.300000,1.000000,9399.232775
75%,51.000000,34.570000,2.000000,17340.746925
max,64.000000,47.740000,5.000000,52589.810300


# 3. Prepare Data

In [12]:
df_cleaned = df.copy()

In [13]:
target = df_cleaned.pop('charges') 

In [14]:
df_cleaned['smoker'].describe()

count     50000
unique        2
top          no
freq      38976
Name: smoker, dtype: object

In [15]:
df_cleaned['sex'].describe()

count     50000
unique        2
top        male
freq      25176
Name: sex, dtype: object

In [16]:
df_cleaned['region'].describe()

count         50000
unique            4
top       southeast
freq          14197
Name: region, dtype: object

In [17]:
df_cleaned['region'].unique()

array(['southeast', 'northeast', 'northwest', 'southwest'], dtype=object)

In [18]:
df_cleaned.info() 

num_cols = ['age', 'bmi', 'children']  
binary_cols = ['sex', 'smoker'] 
cat_cols =['region'] 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       50000 non-null  int64  
 1   sex       50000 non-null  object 
 2   bmi       50000 non-null  float64
 3   children  50000 non-null  int64  
 4   smoker    50000 non-null  object 
 5   region    50000 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 2.3+ MB


In [19]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [20]:
OHE = OneHotEncoder(drop='first', sparse_output=False) 
OHE.set_output(transform='pandas') 

OneHotEncoder(drop='first', sparse_output=False)

## [3.6] One Hot Encode categorical features and save the result in features

In [21]:
OHE_regions = OHE.fit_transform(df_cleaned[['region']]) 

In [22]:
df_cleaned['sex'].unique()

array(['female', 'male'], dtype=object)

In [23]:
sex_binaries = (df_cleaned['sex'] == 'male').astype(float) 
sex_binaries

0        0.0
1        0.0
2        1.0
3        1.0
4        1.0
        ... 
49995    0.0
49996    1.0
49997    1.0
49998    1.0
49999    0.0
Name: sex, Length: 50000, dtype: float64

In [24]:
df_cleaned['smoker'].unique()

array(['no', 'yes'], dtype=object)

In [25]:
smoker_binaries = (df_cleaned['smoker'] == 'yes').astype(float) 
smoker_binaries

0        0.0
1        0.0
2        0.0
3        1.0
4        0.0
        ... 
49995    0.0
49996    0.0
49997    1.0
49998    0.0
49999    0.0
Name: smoker, Length: 50000, dtype: float64

## [3.7] Convert features into a dataframe

In [26]:
df_cleaned.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region'], dtype='object')

In [27]:
num_features = df_cleaned[['age', 'bmi', 'children']] 
features = pd.concat([num_features, sex_binaries, smoker_binaries, OHE_regions], axis=1) 
features

,age,bmi,children,sex,smoker,region_northwest,region_southeast,region_southwest
0,18,33.820,0,0.0,0.0,0.0,1.0,0.0
1,19,23.480,1,0.0,0.0,0.0,1.0,0.0
2,46,30.570,2,1.0,0.0,0.0,1.0,0.0
3,54,32.050,1,1.0,1.0,0.0,1.0,0.0
4,21,21.345,4,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
49995,50,28.880,2,0.0,0.0,0.0,1.0,0.0
49996,58,36.110,2,1.0,0.0,1.0,0.0,0.0
49997,28,24.195,1,1.0,1.0,0.0,0.0,0.0
49998,57,28.600,2,1.0,0.0,0.0,0.0,1.0


In [28]:
print(type(features))

<class 'pandas.core.frame.DataFrame'>


In [29]:
scaler = StandardScaler()
scaler.set_output(transform='pandas') 

StandardScaler()

In [30]:
scaled_features = scaler.fit_transform(features) 

In [31]:
from joblib import dump

In [32]:
print(type(scaler)) 
print(type(OHE))

<class 'sklearn.preprocessing._data.StandardScaler'>
<class 'sklearn.preprocessing._encoders.OneHotEncoder'>


In [33]:
models_dir = os.path.join(project_dir, "models") 

with open(os.path.join(models_dir, "ohe.joblib"), 'wb') as f: 
    dump(OHE, f) 

with open(os.path.join(models_dir, "scaler.joblib"), 'wb') as g: 
    dump(scaler, g) 

print(os.listdir(models_dir))

['ohe.joblib', 'scaler.joblib']


# 4. Split Dataset

In [34]:
from alancarrml.data.sets import split_sets_random

## [4.2] Split the data into training, validation and testing sets

In [35]:
print(type(scaled_features)) 
print(type(target))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [36]:
datasets = split_sets_random(scaled_features, target, test_ratio=0.2) 

X_train = datasets.get('X_train') 
X_val = datasets.get('X_val') 
X_test = datasets.get('X_test') 

y_train = datasets.get('y_train') 
y_val = datasets.get('y_val') 
y_test = datasets.get('y_test') 

In [37]:
print(X_train.shape) 
print(X_val.shape) 
print(X_test.shape)

(30000, 8)
(10000, 8)
(10000, 8)


In [38]:
print(y_train.shape) 
print(y_val.shape) 
print(y_test.shape)

(30000,)
(10000,)
(10000,)


## [4.5] Import the save_sets() function from alancarrml/data/sets.py

In [39]:
from alancarrml.data.sets import save_sets 

In [40]:
datasets_dir = os.path.join(data_dir, 'processed') 
os.path.exists(datasets_dir)

True

In [52]:
save_sets(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test, save_dir=datasets_dir) 
os.listdir(datasets_dir)

['x_val.csv',
 'x_test.csv',
 'y_test.csv',
 'y_train.csv',
 'y_val.csv',
 'x_train.csv']

# 5. Get Baseline Model

In [53]:
from sklearn.dummy import DummyRegressor 

In [54]:
base_reg = DummyRegressor() 

In [58]:
base_reg.fit(X_train, y_train)    

y_base_train = base_reg.predict(X_train) 
y_base_val = base_reg.predict(X_val) 

In [70]:
from alancarrml.models.performance import score_regressors 

train_base_scores = score_regressors(y_base_train, y_train, set_name="base_train") 
val_base_scores = score_regressors(y_base_val, y_val, set_name="base_val")

                Scores for base_train
------------------------------------------------------------
        Root Mean Squared Error    |   12098.0721
        Mean Squared Error         |   146363347.9440
        Mean Absolute Error        |   9099.3816
        R2 Score                   |   0.0000
------------------------------------------------------------

                Scores for base_val
------------------------------------------------------------
        Root Mean Squared Error    |   12179.6721
        Mean Squared Error         |   148344411.6703
        Mean Absolute Error        |   9129.4310
        R2 Score                   |   -0.0001
------------------------------------------------------------

